In [2]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from train.train_model import Train


In [1]:
from train.datasets_preparation import DatasetPreparation
from train.train_model import Train

dataset_preparation = DatasetPreparation()
dataset_preparation.populate_prepared_datasets()

train = Train()
train.start_training()

/home/vozman/projects/slides/slide-analysis-nn/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


GPU not found


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
__________

Epoch 1/200


   1/1037 [..............................] - ETA: 4:06:31 - loss: 0.6928 - acc: 0.5000

...

 219/1037 [=====>........................] - ETA: 2:59:45 - loss: 0.3086 - acc: 0.8756

 220/1037 [=====>........................] - ETA: 2:59:32 - loss: 0.3096 - acc: 0.8751

 221/1037 [=====>........................] - ETA: 2:59:19 - loss: 0.3103 - acc: 0.8750

 222/1037 [=====>........................] - ETA: 2:59:11 - loss: 0.3107 - acc: 0.8750

KeyboardInterrupt: 

In [4]:
from prediction.predict import Predict
predict_example = Predict()

prediction = predict_example.predict_slide('/home/vozman/projects/slides/slide-analysis-nn/train/datasets/source/slide_images/Tumor_001.tif', area_to_predict=((69000, 120000), (75000, 128000)))
prediction.save_as_asap_annotations(truth_xml_path='/home/vozman/projects/slides/slide-analysis-nn/train/datasets/source/slide_images/Tumor_001_true.xml')
print(prediction)

predict
0:23


[1523045030.9629219, 1523045030.9629219, 1523045030.9629219, 1523045030.9629219, 0.0028297901153564453]
est = 324352 days, 4:22:49.730778
1:23
[1523045030.9629219, 1523045030.9629219, 1523045030.9629219, 0.0028297901153564453, 0.00018548965454101562]
est = 232687 days, 10:26:48.723835
2:23


[1523045030.9629219, 1523045030.9629219, 0.0028297901153564453, 0.00018548965454101562, 0.0002276897430419922]
est = 148073 days, 19:44:20.102163
3:23


[1523045030.9629219, 0.0028297901153564453, 0.00018548965454101562, 0.0002276897430419922, 0.001409292221069336]
est = 70511 days, 8:15:23.870296
4:23


[0.0028297901153564453, 0.00018548965454101562, 0.0002276897430419922, 0.001409292221069336, 0.0002491474151611328]
est = 0:00:00.018625
5:23


...

22:23
[3.590416669845581, 3.436448574066162, 3.6167614459991455, 3.6338374614715576, 3.470736026763916]
est = 0:00:03.549640


0:23


[3.436448574066162, 3.6167614459991455, 3.6338374614715576, 3.470736026763916, 44.02540683746338]
est = 0:04:27.642676
1:23


[3.6167614459991455, 3.6338374614715576, 3.470736026763916, 44.02540683746338, 0.0006668567657470703]
est = 0:04:00.888598
2:23


91.70653462409973 sup
Path: /home/vozman/projects/slides/slide-analysis-nn/train/datasets/source/slide_images/Tumor_001.tif
scores: [0.7069279  0.7122552  0.7427167  0.6923453  0.7386433  0.72745925
 0.8034433  0.76480806 0.77936065 0.7955083  0.7986558  0.7912007
 0.79112905 0.7919349  0.78934014 0.7710853  0.7638254  0.7884364
 0.7340335  0.80647594 0.80569464 0.8015797  0.8045274  0.7194962
 0.86209804 0.7732062  0.5229335  0.7235213  0.7524437  0.8032509
 0.80080515 0.791291   0.74259603 0.8005002  0.7868226  0.78290707
 0.78441936 0.78772116 0.7877976  0.79574305 0.80401593 0.79635006
 0.80695057 0.8034313  0.8049745  0.8042805  0.97844464 0.96389204
 0.9022294  0.60064304 0.5957035  0.7138785  0.7704557  0.6297159
 0.58045405 0.72777975 0.7731539  0.78811955 0.8006205  0.79721385
 0.79573953 0.7916186  0.8023631  0.8051594  0.80514336 0.8056194
 0.80394053 0.8060449  0.80056167 0.9830336  0.988986   0.9890911
 0.95046574 0.95664376 0.9392013  0.9045114  0.90282553 0.71812516
 0.8